In [4]:
import conllu
from io import open
import pickle 
import numpy as np

In [5]:
folder = "./UD_Spanish/"
train = "es_ancora-ud-train.conllu"
test = "es_ancora-ud-test.conllu"

In [6]:
train_data = conllu.parse_incr(open(folder+train,'r',encoding = "utf-8"))

In [7]:
c3 = {} # триграммы
c2 = {} # биграммы
c1 = {} # монограммы
cT = {} # пары слово-тег
for token in train_data:
    sentence = [["*","*"]] + [["*","*"]] + [[token[i]["lemma"], token[i]["upostag"]]
        for i in range(len(token)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        c3old = c3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)
        c3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): c3old+1 } ) 

    for i in range(0, len(sentence)-1):
        c2old = c2.get( (sentence[i][1], sentence[i+1][1]), 0)
        c2.update( { (sentence[i][1], sentence[i+1][1]): c2old+1 } )

    for i in range(0, len(sentence)):
        c1old = c1.get( (sentence[i][1]), 0)
        c1.update( { (sentence[i][1]): c1old+1 } )

    for i in range(0, len(sentence)):
        cTold = cT.get( (sentence[i][1], sentence[i][0]), 0)
        cT.update( { (sentence[i][1], sentence[i][0]): cTold+1 } )

In [8]:
print (len(c3), len(c2), len(c1),len(cT))

# Save:
with open('cs_esp.pkl', 'wb') as f:
    pickle.dump([cT, c1, c2, c3], f)

2638 268 20 26499


In [9]:
with open('cs_esp.pkl', 'rb') as f:
    cT, c1, c2, c3 = pickle.load(f)

In [10]:
tags = list(c1.keys())
tags.remove('*')
tags.remove('STOP')
k = len(tags) 

In [11]:

def q(s, u, v):
    eps = 1e-7
    return c3.get((u, v, s), eps) / c2.get((u, v), eps)


def e(x, s):
    eps = 1e-7
    return cT.get((s, x), eps) / c1.get((s), eps)


def K(n): # выдает возможные тэги в зависимости от положения слова в предложении (в начале предложения - *) 
    if n == -1 or n == 0:
        return ['*']
    else:
        return tags

print(K(10))

['DET', 'NOUN', 'ADP', 'ADJ', 'PROPN', 'PRON', 'VERB', 'SCONJ', 'PUNCT', 'AUX', '_', 'CCONJ', 'ADV', 'NUM', 'PART', 'SYM', 'INTJ', 'X']


Алгоритм Витерби

In [12]:
def viterbi(sent):
    Pi = {(0, '*', '*'): 1}
    bp = {}
    n = len(sent) - 2
    y = [""] * (n + 1)

    for k in range(1, n + 1):
        xk = sent[k]
       
        for u in K(k-1):
            for v in K(k):
                w = K(k-2)
                v1 = map(lambda wi:
                            Pi.get((k-1, wi, u)) *
                            q(v, wi, u) *
                            e(xk, v), w)
                v1 = list(v1)
                PiNew = max(v1)
                bpNew = w[v1.index(PiNew)]
                Pi.update({(k, u, v): PiNew})
                bp.update({(k, u, v): bpNew})

    v2 = {}
    for u in K(n-1):
        for v in K(n):
            v2.update({(n, u, v): Pi.get((n, u, v)) * q("STOP", u, v)})

    v2max = max(list(v2.values()))
    for (n, u, v), val in v2.items():
        if val == v2max:
            (y[n-1], y[n]) = (u,v)

    for k in range(n-2, 0, -1):
        y[k] = bp.get((k + 2, y[k+1], y[k+2]))

    print(y[1:])
    return y[1:]

Прогоняем алгоритм на текстовом корпусе

In [13]:
data_test = conllu.parse_incr(open(folder+test,'r',encoding = "utf-8"))
test_tags = [] # оригинальные теги корпуса
predict_tags = [] # предсказанные алгоритмом

for token in data_test:
    sentence=["*"] + [token[i]["lemma"] for i in range(len(token)) ] + ["STOP"]
    test_tags.append([token[i]["upostag"] for i in range(len(token)) ])
    predict_tags.append(viterbi(sentence))

['ADJ', 'ADP', 'DET', 'PUNCT', 'NOUN', 'PUNCT', 'ADP', 'PROPN', 'CCONJ', 'ADP', 'DET', 'PROPN', 'PROPN', 'PUNCT', 'ADP', 'NOUN', 'VERB', 'ADP', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'NOUN', 'ADP', 'NUM', 'NOUN', 'ADP', 'PROPN', 'PROPN', 'CCONJ', 'ADP', 'PROPN', 'ADJ', 'PUNCT', 'CCONJ', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADP', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'NOUN', 'PUNCT']
['NOUN', 'ADV', 'PUNCT', 'DET', 'NUM', 'ADP', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'VERB', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'DET', 'PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'ADP', 'DET', 'NOUN', 'PUNCT', 'CCONJ', 'NUM', 'NOUN', 'ADV', 'ADV', 'AUX', 'VERB', 'NOUN', 'ADP', 'DET', 'PROPN', 'ADJ', 'PUNCT']
['ADV', 'ADP', 'NOUN', 'ADJ', 'ADP', 'DET', 'NOUN', 'PUNCT', 'NOUN', 'ADJ', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADJ', 'ADP', 'NUM', 'ADP', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'DET', 'ADJ', 'PROPN', 'ADP', 'PROPN', 'X', 'PUNCT', 'X', 'PUNCT', 'VERB', '

['CCONJ', 'ADV', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'DET', 'PROPN', 'AUX', 'VERB', 'ADV', 'NOUN', 'SCONJ', 'DET', 'NOUN', 'ADJ', 'ADV', 'PRON', 'AUX', 'VERB', 'ADJ', 'CCONJ', 'ADJ', 'PUNCT', 'ADV', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'PRON', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADP', 'ADJ', 'NOUN', 'PUNCT']
['DET', 'ADJ', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'SCONJ', 'VERB', 'PUNCT', 'CCONJ', 'ADV', 'DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADJ', 'PUNCT']
['DET', 'PROPN', 'VERB', 'ADV', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADP', 'DET', 'PRON', 'ADV', 'ADV', 'AUX', 'VERB', 'ADP', 'ADJ', 'NOUN', 'ADJ', 'PUNCT']
['PRON', 'ADJ', 'ADP', 'NOUN', 'PUNCT', 'ADP', 'DET', 'X', 'ADV', 'X', 'PUNCT', 'X', 'CCONJ', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'PUNCT', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT']
['DET', 'NOUN', 'AUX', 'ADJ', 'PUNCT', 'ADJ', 'CCONJ', 'ADJ', 'PUNCT']
['DET', 'NOUN', 'ADP', 'NUM', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN',

['ADV', 'ADV', 'DET', 'NOUN', 'PRON', 'VERB', 'ADP', 'INTJ', 'X', 'CCONJ', 'X', 'NOUN', 'X', 'PUNCT', 'ADJ', 'ADP', 'DET', 'PRON', 'VERB', 'PUNCT', 'ADV', 'PUNCT', 'DET', 'NOUN', 'ADP', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT']
['ADP', 'DET', 'NOUN', 'PUNCT', 'SCONJ', 'AUX', 'VERB', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'VERB', 'ADV', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'CCONJ', 'VERB', 'ADV', 'ADP', 'DET', 'NOUN', 'PUNCT']
['SCONJ', 'PUNCT', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'PRON', 'DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADP', 'NOUN', 'PUNCT']
['ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NUM', 'ADJ', 'NOUN', 'PUNCT', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'ADV', 'VERB', 'DET', 'ADV', 'ADJ', 'NOUN', 'PUNCT']
['CCONJ', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADV', 'ADP', 'PRON', 'ADP', 'DET', 'NOUN', 'VERB', 'SCONJ', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'PRON', 'VERB', 'PUNCT']
['PRON', 'ADV', 'PRON', 'VERB', 'ADP', 'VERB

['DET', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'PUNCT', 'AUX', 'SCONJ', 'DET', 'NOUN', 'ADJ', 'ADJ', 'VERB', 'ADP', 'DET', 'NOUN', 'ADV', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'PUNCT', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'DET', 'PRON', 'ADP', 'PRON', 'ADV', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT']
['ADV', 'VERB', 'ADV', 'DET', 'NOUN', 'CCONJ', 'NOUN', 'ADV', 'PRON', 'VERB', 'ADV', 'ADV', 'DET', 'NOUN', 'PUNCT', 'PUNCT', 'PRON', 'AUX', 'ADJ', 'SCONJ', 'DET', 'NOUN', 'ADJ', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADV', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'PRON', 'ADP', 'DET', 'NOUN', 'PUNCT']
['DET', 'NOUN', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'ADP', 'NOUN', 'PROPN', 'PROPN', 'ADP', 'PROPN', 'PUNCT', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PUNCT', 'ADV', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADV', 'AUX', 'ADJ', 'CCONJ', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET'

['NOUN', 'ADJ', 'PRON', 'VERB', 'ADV', 'ADV', 'ADP', 'DET', 'NOUN', 'PUNCT', 'SCONJ', 'DET', 'NOUN', 'AUX', 'AUX', 'ADJ', 'CCONJ', 'ADP', 'NOUN', 'ADJ', 'PUNCT', 'ADP', 'DET', 'PRON', 'DET', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'AUX', 'VERB', 'SCONJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT']
['ADJ', 'ADP', 'PRON', 'CCONJ', 'DET', 'NOUN', 'DET', 'NOUN', 'AUX', 'PUNCT', 'VERB', 'PROPN', 'PUNCT', 'ADJ', 'SCONJ', 'DET', 'NOUN', 'ADP', 'DET', 'PRON', 'AUX', 'VERB', 'PUNCT', 'CCONJ', 'ADV', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'DET', 'NOUN', 'ADJ', 'ADP', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT']
['NUM', 'NOUN', 'VERB', 'PUNCT', 'SCONJ', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'ADJ', 'VERB', 'DET', 'NOUN', 'ADJ', 'PUNCT']
['VERB', 'ADP', 'NOUN', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'CCONJ', 'DET', 'ADP', 'NOUN', 'AUX', 'ADJ', 'ADV', 'VERB', 'DET', 'ADJ', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'CCONJ', 'ADP', 'N

...

['DET', 'NOUN', 'PRON', 'VERB', 'SCONJ', 'DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'PUNCT']
['ADP', 'NOUN', 'ADJ', 'PUNCT', 'NOUN', 'ADP', 'PRON', 'ADJ', 'ADP', 'NOUN', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'VERB', 'DET', 'PRON', 'VERB', 'PUNCT', 'PRON', 'PRON', 'VERB', 'ADP', 'NOUN', 'ADP', 'DET', 'PRON', 'PRON', 'VERB', 'DET', 'NOUN', 'PUNCT', 'PRON', 'VERB', 'ADV', 'PUNCT']
['DET', 'NOUN', 'VERB', 'SCONJ', 'SCONJ', 'DET', 'NOUN', 'AUX', 'ADP', 'VERB', 'PUNCT', 'ADV', 'AUX', 'VERB', 'DET', 'NOUN', 'ADP', 'NUM', 'PUNCT']
['DET', 'NOUN', 'VERB', 'NUM', 'NOUN', 'ADP', 'NUM', 'NOUN', 'PUNCT']
['DET', 'NOUN', 'ADJ', 'ADP', 'PROPN', 'PROPN', 'PUNCT', 'PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'PUNCT', 'VERB', 'ADP', 'DET', 'NOUN', 'DET', 'NOUN', 'ADJ', 'ADP', 'NUM', 'NOUN', 'PUNCT', 'ADP', 'PROPN', 'PROPN', 'PUNCT']
['ADP', 'DET', 'NUM', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'ADP', 'DET', 'ADJ', 'NUM', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'PUNCT', 'DET',

['ADP', 'NOUN', 'PUNCT', 'PART', 'ADV', 'ADP', 'DET', 'PROPN', 'ADP', 'DET', 'PROPN', 'PROPN', 'PUNCT', 'VERB', 'ADV', 'DET', 'NOUN', 'PROPN', 'PROPN', 'PROPN', 'CCONJ', 'PROPN', 'PROPN', 'PUNCT']
['DET', 'NOUN', 'ADP', 'PROPN', 'ADP', 'NOUN', 'PUNCT']
['DET', 'SYM', 'NOUN', 'SYM', 'VERB', 'SCONJ', 'DET', 'NOUN', 'ADJ', 'AUX', 'VERB', 'DET', 'NOUN', 'ADP', 'PROPN', 'ADP', 'PROPN', 'PUNCT']
['DET', 'NOUN', 'ADP', 'DET', 'PROPN', 'PUNCT', 'PROPN', 'PROPN', 'PUNCT', 'VERB', 'ADV', 'ADP', 'DET', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADV', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'SCONJ', 'DET', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADV', 'AUX', 'DET', 'NOUN', 'PUNCT', 'CCONJ', 'DET', 'NOUN', 'ADJ', 'CCONJ', 'DET', 'NOUN', 'PRON', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'ADP', 'PROPN', 'PROPN', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'DET', 'NOUN', 'ADJ', 'PUNCT']
['PUNCT', 'DET', 'ADV', 'ADJ', 'AUX', 'SCONJ', 'AUX', 'VERB', 'SCONJ', 'DET', '

['ADP', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'AUX', 'ADJ', 'CCONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'PUNCT']
['ADP', 'PRON', 'AUX', 'VERB', 'DET', 'NUM', 'NOUN', 'ADJ', 'ADP', 'DET', 'NOUN', 'PUNCT']
['ADP', 'PROPN', 'PROPN', 'DET', 'NOUN', 'ADV', 'PRON', 'VERB', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN', 'ADP', 'PROPN', 'PUNCT']
['DET', 'NOUN', 'ADJ', 'VERB', 'DET', 'NOUN', 'ADJ', 'DET', 'NOUN', 'ADP', 'NOUN', 'NUM', 'PUNCT']
['DET', 'PROPN', 'AUX', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT']
['DET', 'NOUN', 'ADJ', 'ADP', 'PROPN', 'VERB', 'ADV', 'ADV', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'PUNCT']
['DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADV', 'AUX', 'AUX', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'PROPN', 'PUNCT']
['ADV', 'AUX', 'ADJ', 'SCONJ', 'PRON', 'VERB', 'DET', 

['DET', 'NOUN', 'ADJ', 'VERB', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'DET', 'NOUN', 'ADV', 'ADP', 'PROPN', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADV', 'ADJ', 'PUNCT']
['DET', 'NOUN', 'ADJ', 'PRON', 'AUX', 'VERB', 'DET', 'NOUN', 'ADP', 'VERB', 'ADP', 'DET', 'NOUN', 'DET', 'NOUN', 'ADJ', 'ADP', 'VERB', 'ADP', 'DET', 'NOUN', 'PRON', 'ADP', 'DET', 'PROPN', 'ADP', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT']
['PUNCT', 'DET', 'NOUN', 'ADJ', 'ADV', 'AUX', 'ADV', 'ADJ', 'ADP', 'DET', 'PROPN', 'CCONJ', 'DET', 'NOUN', 'VERB', 'SCONJ', 'AUX', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'CCONJ', 'VERB', 'SCONJ', 'DET', 'NOUN', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADV', 'DET', 'NOUN', 'PUNCT', 'PUNCT', 'VERB', 'PROPN', 'PROPN', 'PUNCT']
['DET', 'NOUN', 'ADP', 'PROPN', 'ADP', 'PROPN', 'PROPN', 'PUNCT', 'SCONJ', 'PUNCT', 'SCONJ', 'PRON', 'PUNCT', 'VERB', 'ADP', 'PROPN', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'AUX', 'VERB', 'ADP', 'NOUN', '

['DET', 'NOUN', 'ADJ', 'PUNCT', 'CCONJ', 'DET', 'NOUN', 'PUNCT', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADJ', 'ADP', 'DET', 'PROPN', 'PROPN', 'ADP', 'PROPN', 'CCONJ', 'ADJ', 'ADP', 'DET', 'PROPN', 'ADP', 'NOUN', 'PUNCT']
['DET', 'NOUN', 'ADV', 'VERB', 'NOUN', 'ADJ', 'ADP', 'DET', 'PROPN', 'CCONJ', 'ADP', 'NOUN', 'ADJ', 'PUNCT']
['DET', 'NOUN', 'VERB', 'ADV', 'SCONJ', 'PRON', 'AUX', 'AUX', 'VERB', 'DET', 'ADJ', 'ADP', 'SYM', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADP', 'PROPN', 'PUNCT']
['PART', 'ADV', 'AUX', 'AUX', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PUNCT']
['PUNCT', 'AUX', 'DET', 'SYM', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'PUNCT', 'PUNCT', 'VERB', 'PUNCT']
['VERB', 'PUNCT', 'ADV', 'PUNCT', 'SCONJ', 'PRON', 'AUX', 'ADP', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADP', 'PROPN', 'PUNCT', 'PRON', 'SCONJ', 'AUX', 'VERB', 'DET', 'ADJ', 'NOUN', 'PUNCT']
['PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'PROPN', 'ADP', 'VERB', 'PUNCT', 'NOUN', 'PUNCT', 'ADP', 'NOUN', 'PROPN', 'PROPN', 'PUNCT', '

Считаем ошибку

In [14]:
err = 0.
for sent in range(len(test_tags)):
    err += np.mean(np.array(test_tags[sent]) != np.array(predict_tags[sent]))
print('Error = ', err/len(test_tags))

Error =  0.04965178740458406
